# Filtro de spam


#### [Entrenamiento:](#Entrenamiento)
- def que separe una línea en palabras y las agregue a un conjunto dado por parámetro
- def de un generator que devuelva un set de palabras por cada correo de un mbox dado por parámetro
- def que devuelve Dict de la forma {'w' : n } con w = palabra y n = Hw para 'ham.mbox' o n = Sw para 'spam.box'

#### [Clasificación:](#Clasificación)
- def que devuelva P(y|xw) para cada palabra w de un conjunto de palabras, uno por cada correo de 'news.box'
- def que devuelva lista de 15 palabras de mejor clasificación individual de un correo nuevo
- def que devuelva P(y|x1, . . . , x15) de la lista anterior
- def que devuelva lista de 0 o 1 por cada correo de 'news.mbox' en el mismo orden

#### [Incorporación:](#Incorporación)
- def que devuelva un set de palabras de un 'new-spam.mbox' o 'new-ham.mbox' que contiene un solo mensaje
- def que combine el anterior conjunto al dict de entrenamiento aumentando en 1 cada valor
- def que escriba el nuevo correo 'new-spam.mbox' o 'new-ham.mbox' en el fichero que corresponda

#### [Uso:](#Uso)
- Ejemplo de uso de los 3 procedimientos anteriores

# Entrenamiento

In [ ]:
import re


# Separa al string 'line' según el regex delimiter
# Y añade los trozos al set 'words' pasado por parámetro
def process_line(line, words, delimiter = '[,. ()]|\n|\t'):
    for word in re.split(delimiter, line):
        words.add(word)


# Devuelve un set de palabras diferentes de un solo correo
def words_per_mail(mbox):    
    with open(mbox) as fp:
        lines = iter(fp.readlines())
        while True:
            words = set()
            for subject in lines:
                if subject.startswith("Subject: "):
                    break
            else:
                break

            process_line(subject[9:], words)

            for status in lines:
                if status == "X-KMail-MDN-Sent:  \n":
                    break

            intro = fromOne = False
            for line in lines:
                if line == '\n' :
                    intro = True
                    continue
                elif line.startswith('From: ') and fromOne :
                    break
                elif line.startswith('From ') and intro :
                    fromOne = True
                else:
                    intro = False
                    process_line(line, words)

            words.remove('')
            yield words


# Agrega words a dictionary, incrementando la ocurrencia en 1
def merge_to_dict(words, dictionary):
    for w in words:
        dictionary[w] = dictionary[w] + 1 if w in dictionary else 1


# Devuelve dict de palabras no repetidas y el número de correos
# en los que aparecen, del mbox dado por parámetro
def training_dict(mbox):
    dictionary = {}
    for words in words_per_mail(mbox):
        merge_to_dict(words, dictionary)
    return dictionary


In [ ]:
# Devuelve dict de la forma {'word' : Sw } para spambox
# y otro dict de la forma {'word' : Hw } para hambox
def training(spambox, hambox):
    return training_dict(spambox), training_dict(hambox)


# Clasificación

In [ ]:
# Devuelve generator que recorre las palabras de cada correo de newsbox
# Devuelve un set por cada llamada, que corresponde al siguiente correo
def news_words_generator(newsbox = './news.mbox'):
    return words_per_mail(newsbox)


# Incorporación

In [ ]:
# Escribe el nuevo correo ubicado en la ruta 'new'
# en el fichero de ruta 'box'
def append_new_to_mbox(new, mbox):
    with open(mbox, 'a') as writable:
        with open(new) as readable:
            writable.write(readable.read())


# Actualiza dictionary a partir del set de palabras
# del único mensaje contenido en 'new_box'
def update_dict(new_box, dictionary):
    new_words = next(words_per_mail(new_box))
    merge_to_dict(new_words, dictionary)


In [ ]:
# Actualiza dictionary y escribe el contenido de new_box en mbox
def incorporation(dictionary, new_box, mbox):
    update_dict(new_box, dictionary)
    append_new_to_mbox(new_box, mbox)


# Uso

In [ ]:
# Archivos
newspam = './new-spam.mbox'
newham = './new-ham.mbox'
spambox = './spam.mbox'
hambox = './ham.mbox'

# Entrenamiento
spam_dict, ham_dict = training(spambox, hambox)

# Clasificación
# Falta

# Incorporación a spam.box
incorporation(spam_dict, newspam, spambox)

# Incorporación a ham.box
incorporation(ham_dict, newham, hambox)